### Import Dependencies

In [2]:
import sys
sys.path.append("../")
import pandas as pd
import logging

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown
from utils.json_utils import CrisisEvent,pydantic_to_json_schema,parse_json_with_pydantic,validate_json_schema,safe_parse_json

### Loading data from News feed.txt file

In [3]:
file_path = '../data/raw/News feed.txt'
news=[]

with open(file_path, 'r', encoding='utf-8') as f:
    news = [line.strip() for line in f if line.strip()]

print(news)

['BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.', 'SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boat immediately.', 'Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims reported.', 'URGENT: Landslide in Kalutara. 12 people missing. Rescue team needed.', 'Gampaha town center is fully underwater. Flood level est 2.0 meters. 500 people displaced to temple. Need dry rations.', 'Just saw a navy boat in Colombo. Good job guys.', 'Matara reports heavy rain but no floods yet. Status stable.', 'Help! My grandmother is stuck in Beddagana (Colombo). She is 80 years old. Water level 1.5m.', 'Galle fort area is safe. Tourists are fine.', 'Warning: Dengue risk rising in Gampaha due to stagnant water.', 'SOS: Kaduwela highway entrance (Colombo) blocked. Bus trapped with 40 passengers. Need evacuation.', 'Donation drive starting at Town Hall. We need water bottles and biscuits.', 'Akura

### Extracting json data from  json_extract.v1

In [4]:
schema=pydantic_to_json_schema(CrisisEvent)
prompt_text, spec = render(
    "json_extract.v1",
    schema=schema,
    text="{text}"
)

In [5]:
model = pick_model('openai', 'general')
client = LLMClient('openai', model)

valid_records = []
for new in news:
    full_prompt = prompt_text.replace("{text}", new)
    msg_payload = [{'role': 'user', 'content': full_prompt}]
    
    response = client.chat(msg_payload, temperature=0.0)
    raw_text = response['text'].strip()
    try:
        validated_obj = CrisisEvent.model_validate_json(raw_text)
        valid_records.append(validated_obj.model_dump())
        print(f"Success: {validated_obj.district}")
        
    except Exception as e:
        print(f"Validation Error in line: {new[:30]}... | Error: {e}")
    

Success: Colombo
Success: Gampaha
Success: Kandy
Success: Kalutara
Success: Gampaha
Success: Colombo
Success: Matara
Success: Colombo
Success: Galle
Success: Gampaha
Success: Colombo
Success: Colombo
Success: Kandy
Success: Gampaha
Success: Nuwara Eliya
Success: Gampaha
Success: Ratnapura
Success: Colombo
Success: Kalutara
Success: Colombo
Success: Gampaha
Success: Kandy
Success: Gampaha
Success: Kalutara
Success: Matara
Success: Colombo
Success: Galle
Success: Gampaha
Success: Colombo
Success: Kegalle


### Convert valid objects to a Pandas DataFrame and save as lood_report.xlsx.

In [6]:
df = pd.DataFrame(valid_records)
df.head()

,district,flood_level_meters,vicm_count,main_need,status
0,Colombo,9.5,0,General Assistance,Critical
1,Gampaha,NaN,5,Rescue,Critical
2,Kandy,NaN,0,General Assistance,Stable
3,Kalutara,NaN,12,Rescue,Critical
4,Gampaha,2.0,500,Food/Water,Critical


In [9]:
df.to_excel('../data/output/flood_report.xlsx', index=False)